---
## Step 0: Import Datasets

In [8]:
import data

classes, train_labels, train_files  = data.load_dataset('./data/train')
_, test_labels, test_files  = data.load_dataset('./data/test')
_, valid_labels, valid_files  = data.load_dataset('./data/valid')

print('Train data count: {}'.format(len(train_labels)))
print('Test data count: {}'.format(len(test_labels)))
print('Valid. data count: {}'.format(len(valid_labels)))
print('Classes: {}'.format(classes))

Train data count: 2000
Test data count: 600
Valid. data count: 150
Classes: {'seborrheic_keratosis': 0, 'nevus': 2, 'melanoma': 1}


### Use numeric targets instead of text labels

In [9]:
train_targets = [classes[label] for label in train_labels]
test_targets = [classes[label] for label in test_labels]
valid_targets = [classes[label] for label in valid_labels]

target_to_label = {target: label for label, target in classes.items()}
print(target_to_label)

{0: 'seborrheic_keratosis', 1: 'melanoma', 2: 'nevus'}


---
## Step 1 Load pretrained Inception v3 model

In [10]:
import torch
import torchvision.models

# are we GPU capable?
use_gpu = torch.cuda.is_available()

inception_model= torchvision.models.inception_v3(pretrained=True)
if use_gpu:
    inception_model = inception_model.cuda()

### Create single image loader
For further use we create a image loader from the files. Here we need an image normalizer, so all image pixel values are floats and in a range between -1 and 1.  
Next we create a loader that composes several image transformations: resize, center crop, normalization and conversion to PyTorch tensor.  
Inception V3 requires the image size of 299. 

In [11]:
from torchvision import datasets, models, transforms
from torch.autograd import Variable
from PIL import Image

imsize = 299

# create image normalizer
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225])

# create image transformer
image_transform = transforms.Compose([
    transforms.Resize(imsize),
    transforms.CenterCrop(imsize),
    transforms.ToTensor(),
    normalize])

def image_loader(img_path):
    '''
    Load image from the file and transform it to be consumed by the model
    
    :param img_path: path to the image file
    :return: PyTorch 4D tensor - (1, channels, width, height). CPU or GPU tensor
    '''
    image = Image.open(img_path)
    image = image_transform(image).float()
    image = Variable(image, requires_grad=False)
    image = image.unsqueeze(0)  # to have 4D tensor
    if use_gpu:
        image = image.cuda()
    return image

### Predict function
Now we create a prediction function for the single image. First we load the image and then run prediction on it using inception model.  
  
**Caution**  
Since Inception model uses batch normalization, we must call ```model.eval()``` method to let the model know that we are in prediction mode.

In [12]:
def predict_labels(img_path):
    '''
    Predicts an integer labels on the image
    
    :param img_path: path to the image file
    :return: one of the 1000 pretrained labels on the ImageNet data set
    '''
    image = image_loader(img_path)
    inception_model.eval()
    outputs = inception_model(image)
    _, pred_idx = torch.max(outputs.data, 1)
    return pred_idx[0]

---
## Step 3: Data set and its loader

### Skin cancer dataset class to provide images and their labels
First of all, we define a class to that helps to iterate over the images and their labels. It is a PyTorch specialty, later it is used by the data loader.

In [13]:
from torch.utils.data import Dataset, DataLoader

class SkinCancerDataset(Dataset):
    def __init__(self, image_files, targets, transform=None):
        self.image_files = image_files
        self.targets = targets
        self.transform = transform
        
    def __len__(self):
        assert(len(self.targets) == len(self.image_files))
        return len(self.targets)
    
    def __getitem__(self, idx):
        image = Image.open(self.image_files[idx])
        if self.transform:
            image = self.transform(image)
        targets = self.targets[idx]
        return [image, targets]

### Loader of images and labels
Here we create train, validation and test datasets and loaders for them.

In [14]:
train_ds = SkinCancerDataset(train_files, train_targets, transform=image_transform)
test_ds = SkinCancerDataset(test_files, test_targets, transform=image_transform)
valid_ds = SkinCancerDataset(valid_files, valid_targets, transform=image_transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_ds, batch_size=16, shuffle=False)
valid_loader = DataLoader(valid_ds, batch_size=32, shuffle=True, num_workers=4)

### Test the loader
Here is a short test of the interplay between data set, data loader and model. Prediction by Inception and targets are not match here, since we have only 3 types of skin cancer, but Inception has 1000 classes as an output.

In [15]:
def get_inputs_targets_tensors(inputs, targets):
    '''
    Helper function for PyTorch tensor creation
    
    :param inputs: inputs (such as images)
    :param targets: target classes
    :return: tuple of inputs and targets PyTorch tensors
    '''
    inputs = Variable(inputs)
    targets = Variable(targets)
    if use_gpu:
        inputs = inputs.cuda()
        targets = targets.cuda()
    
    return inputs, targets

In [16]:
inputs, targets = next(iter(test_loader))

inputs, targets = get_inputs_targets_tensors(inputs, targets)

inception_model.eval()
outputs = inception_model(inputs)
_, pred_idx = torch.max(outputs.data, 1)

print(pred_idx)
print(targets.data)


 646
  78
 964
 107
 314
 640
 948
 769
 419
 961
  78
 551
 769
 798
  78
  78
[torch.cuda.LongTensor of size 16 (GPU 0)]


 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.cuda.LongTensor of size 16 (GPU 0)]



---
## Step 4: Modify and train Inception model
In order to predict the do breeds only, we need to replace a last fully-connected layer of the Inception model (which predict one of the 1000 categories) with a fully-connected layer for the cancer skin prediction (which has 3 categories). Then we train weights only for this new layer.

### Helpers
We need helper functions to prepare and save the checkpoints during the training. It allows us to save the intermediate results and the best model.

In [17]:
import shutil
import os

checkpoints_dir = './checkpoints/'
best_model_filename = 'model_best.pth.tar'
checkpoint_filename = 'checkpoint.pth.tar'

def best_model_exists():
    '''
    Defines whether the file with a best model exists
    
    :return: True if the best model file exists, False - otherwise
    '''
    best_model_checkpoint = checkpoints_dir + best_model_filename
    return os.path.exists(best_model_checkpoint)

def load_saved_best_weights(model):
    '''
    Loads the saved best weights for the model
    
    :param model: model to load the weights for
    '''
    best_model_checkpoint = checkpoints_dir + best_model_filename
    model.load_state_dict(torch.load(best_model_checkpoint))

def prepare_checkpoints_dir():
    '''
    Prepares the checkpoints directory - removes old results
    '''
    if os.path.exists(checkpoints_dir):
        shutil.rmtree(checkpoints_dir)
    os.makedirs(checkpoints_dir)

def save_checkpoint(model, is_best, filename=checkpoint_filename):
    '''
    Saves checkpoint into the file
    
    :param model: model to save
    :param is_best: True if the model is a best ones, False - otherwise
    :param filename: checkpoint file name
    '''
    filepath = checkpoints_dir + filename
    torch.save(model.state_dict(), filepath)
    if is_best:
        best_filepath = checkpoints_dir + best_model_filename
        shutil.copyfile(filepath, best_filepath)

### Epoch train function
Train function for a single epoch. It runs the training on the batches of a single epoch. It is called multiple times during the training.

In [18]:
def train_epoch(model, epoch, loader, criterion, optimizer):
    '''
    Trains the model during the single epoch
    
    :param model: model to train
    :param epoch: epoch number
    :param loader: PyTorch data loader
    :param criterion: calculates the loss
    :param optimizer: model optimizer such as Adam, SGD, etc.
    :return: tuple of train loss and accuracy
    '''
    dataset_size = len(loader.dataset)
    running_loss = 0.0
    running_correct = 0
    log_after = 20
    batch_counter = 0
    
    model.train()
    for batch_idx, (inputs, targets) in enumerate(loader):
        inputs, targets = get_inputs_targets_tensors(inputs, targets)

        optimizer.zero_grad()
        outputs = model(inputs)
        
        # inception model has 2 outputs during the training (returned as tuple):
        # predicted class and aux logits. we need here only the first
        _, pred_idx = torch.max(outputs[0].data, 1)

        loss = criterion(outputs[0], targets)
        loss.backward()
        optimizer.step()

        if (batch_counter % log_after) == 0:
            print('Train Epoch: {} [{}/{}]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(inputs), 
                dataset_size, loss.data[0]))
        running_loss += loss.data[0]
        running_correct += torch.sum(pred_idx == targets.data)
        batch_counter += 1
    
    epoch_loss = running_loss / dataset_size
    epoch_acc = running_correct / dataset_size
    
    return (epoch_loss, epoch_acc)

### Epoch test function
Test function for a single epoch. It is called multiple times during the training and once during the tests.

In [19]:
def test_epoch(model, epoch, loader):
    '''
    Tests the model during the single epoch
    
    :param model: model to test
    :param epoch: epoch number
    :param loader: PyTorch data loader
    :return: tuple of test loss and accuracy
    '''
    dataset_size = len(loader.dataset)
    running_loss = 0.0
    running_correct = 0
    log_after = 10
    batch_counter = 0
    
    model.eval()
    for batch_idx, (inputs, targets) in enumerate(loader):
        inputs, targets = get_inputs_targets_tensors(inputs, targets)

        outputs = model(inputs)
        _, pred_idx = torch.max(outputs.data, 1)
        loss = criterion(outputs, targets)

        if (batch_counter % log_after) == 0:
            print('Test Epoch: {} [{}/{}]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(inputs), 
                dataset_size, loss.data[0]))
        
        running_loss += loss.data[0]
        running_correct += torch.sum(pred_idx == targets.data)
        batch_counter += 1
        
    epoch_loss = running_loss / dataset_size
    epoch_acc = running_correct / dataset_size
    
    return (epoch_loss, epoch_acc)

### Train function
Here is a model train function. We iterate over several epoch and call single epoch train and test for each iteration. Using the test accuracy the function determines the best model, it also saves the checkpoints for each epoch and prints te results.

In [20]:
def train(model, train_loader, valid_loader, criterion, optimizer, epochs=10):
    '''
    Trains the model, saves checkpoints and the best model
    
    :param model: model to train
    :param train_loader: PyTorch train data loader
    :param test_loader: PyTorch test data loader
    :param criterion: calculates the loss
    :param optimizer: model optimizer such as Adam, SGD, etc.
    :return: trained model
    '''
    best_model_wts = model.state_dict()
    best_acc = 0.0
    best_loss = 1000.0
    
    for epoch in range(1, epochs + 1):
        train_epoch_loss, train_epoch_acc = train_epoch(model, epoch, train_loader, criterion, optimizer)
        valid_epoch_loss, valid_epoch_acc = test_epoch(model, epoch, valid_loader)
            
        is_best = False
        if valid_epoch_acc > best_acc:
            best_acc = valid_epoch_acc
            best_loss = valid_epoch_loss
            best_model_wts = model.state_dict()
            is_best = True
            
        save_checkpoint(model, is_best)
        
        print('Epoch [{}/{}]\ttrain loss: {:.4f}\ttrain acc: {:.4f}\tvalid loss: {:.4f}\tvalid acc: {:.4f}'.format(
                epoch, epochs,
                train_epoch_loss, train_epoch_acc, 
                valid_epoch_loss, valid_epoch_acc))
            
    print('Best validation accuracy: {:6f}\tBest validation loss: {:6f}'.format(best_acc, best_loss))

    model.load_state_dict(best_model_wts)
    return model

### Modify Inception model
Here we modify the Inception model for training and prediction of the skin cancer. At the end we create a loss function (Cross Entropy Loss) and use Adam optimizer for the model training.

In [21]:
# load pretrained Inception model
inception = torchvision.models.inception_v3(pretrained=True)

# freeze all model parameters
for param in inception.parameters():
    param.requires_grad = False
    
# new final layer with 3 classes
num_features = inception.fc.in_features
inception.fc = torch.nn.Linear(num_features, len(classes))

if use_gpu:
    inception = inception.cuda()

# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(inception.fc.parameters(), lr = 0.001)

In [22]:
print(inception)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d (3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d (32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d (32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d (64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d (80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True)
  )
  (Mixed_5b): InceptionA(
    (branch1x1): BasicConv2d(
      (conv): Conv2d (192, 64, kernel_si

Now we can load the previously saved model or start the model training.

In [23]:
# avoid the PIL error "OSError: image file is truncated (150 bytes not processed)"
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

if best_model_exists():
    final_model = inception
    load_saved_best_weights(final_model)
else:
    prepare_checkpoints_dir()
    final_model = train(inception, train_loader, valid_loader, criterion, optimizer, epochs=10)

---
## Step 5: Test the model

### Helper function to predict skin cancer on the speicified image

In [24]:
def predict_skin_cancer(img_path):
    '''
    Predicts the skin cancer type on the image
    
    :param img_path: path to the image file
    :return: predicted name of the dog breed
    '''
    image = image_loader(img_path)
    
    final_model.eval()
    outputs = final_model(image)
    _, pred_idx = torch.max(outputs.data, 1)
    
    return target_to_label[pred_idx[0]]

### Test model on test data

In [18]:
correct = 0

final_model.eval()
for inputs, targets in test_loader:
    inputs, targets = get_inputs_targets_tensors(inputs, targets)

    outputs = final_model(inputs)
    _, pred_idx = torch.max(outputs.data, 1)

    correct += torch.sum(pred_idx == targets.data)

accuracy = correct / len(test_loader.dataset)
print('Test accuracy: {:6f}'.format(accuracy))

Test accuracy: 0.661667


### Read sample prediction CSV, run prediction on the images and save results into a final CSV

In [30]:
import pandas as pd

dt = pd.read_csv('sample_predictions.csv', index_col=False)
dt_res = pd.DataFrame(columns=['Id', 'task_1', 'task_2'])
for i, row in dt.iterrows():
    res = {}
    
    pred = predict_skin_cancer(row['Id'])
    res['Id'] = row['Id']
    
    task_1 = 0
    if pred == 'melanoma':
        task_1 = 1
    
    task_2 = 0
    if pred == 'seborrheic_keratosis':
        task_2 = 1
        
    dt_res.loc[i] = [row['Id'], task_1, task_2]

print(dt_res)
dt_res.to_csv('final_predictions.csv', index=False)

                                                  Id task_1 task_2
0                data/test/melanoma/ISIC_0012258.jpg      0      0
1                data/test/melanoma/ISIC_0012356.jpg      0      1
2                data/test/melanoma/ISIC_0012369.jpg      1      0
3                data/test/melanoma/ISIC_0012395.jpg      0      1
4                data/test/melanoma/ISIC_0012425.jpg      0      0
5                data/test/melanoma/ISIC_0012758.jpg      0      0
6                data/test/melanoma/ISIC_0012989.jpg      0      0
7                data/test/melanoma/ISIC_0013072.jpg      1      0
8                data/test/melanoma/ISIC_0013073.jpg      1      0
9                data/test/melanoma/ISIC_0013242.jpg      1      0
10               data/test/melanoma/ISIC_0013277.jpg      0      0
11               data/test/melanoma/ISIC_0013321.jpg      0      1
12               data/test/melanoma/ISIC_0013374.jpg      1      0
13               data/test/melanoma/ISIC_0013411.jpg      1   